<a href="https://colab.research.google.com/github/Tiabet/Complete_story/blob/master/TF_KoGPT_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture

!pip install transformers
!pip install accelerate

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import torch
import tensorflow as tf

In [ ]:
# Load model directly
from transformers import AutoTokenizer, TFGPT2LMHeadModel, GPT2Config

tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

# Add special tokens, including [SEP]
special_tokens = {'bos_token': '</s>', 'eos_token': '</s>', 'pad_token': '<pad>', 'sep_token': '<sep>', 'unk_token' : '<unk>'}
tokenizer.add_special_tokens(special_tokens)


# Load the existing model checkpoint
model_config = GPT2Config.from_pretrained("Tiabet/Tiabet-tensorflow-finetuned-koGPT-complete_story-epoch-3")

# Add the new special token to the model's config

model_config.sep_token_id = tokenizer.sep_token_id  # Add this line


# Initialize the model
model = TFGPT2LMHeadModel.from_pretrained("Tiabet/Tiabet-tensorflow-finetuned-koGPT-complete_story-epoch-3", config=model_config)

model.resize_token_embeddings(len(tokenizer))

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at Tiabet/Tiabet-tensorflow-finetuned-koGPT-complete_story-epoch-3.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
import json

def jsonlload(fname):
    with open(fname, "r", encoding="utf-8") as f:
        lines = f.read().strip().split("\n")
        j_list = [json.loads(line) for line in lines]

    return j_list


def jsonldump(j_list, fname):
    with open(fname, "w", encoding='utf-8') as f:
        for json_data in j_list:
            f.write(json.dumps(json_data, ensure_ascii=False)+'\n')


In [ ]:
j_list = jsonlload("/content/drive/MyDrive/nikluge-sc-2023-test.jsonl")

In [ ]:
import pandas as pd
import tqdm

# Initialize empty lists to store data
sentence1_list = []
sentence3_list = []

# Read the JSONL file line by line and extract data
with open("/content/drive/MyDrive/nikluge-sc-2023-test.jsonl", 'r', encoding='utf-8') as file:
    for line in file:
        data = eval(line)  # Evaluate the JSONL line as a Python dictionary
        sentence1 = data['input']['sentence1']
        sentence3 = data['input']['sentence3']
        sentence1_list.append(sentence1)
        sentence3_list.append(sentence3)


# Create a Pandas DataFrame
test_data = pd.DataFrame({'sentence1': sentence1_list, 'sentence3': sentence3_list})

In [ ]:
batch_size = 256

steps = len(test_data) // batch_size + 1

def get_test_data():
    for sentence1, sentence3 in zip(test_data.sentence1.to_list(), test_data.sentence3.to_list()):
        bos_token = [tokenizer.bos_token_id]
        sent = tokenizer.encode('<usr>' + sentence1 + '<sep>' + sentence3 + '<sys>')
        yield bos_token + sent

# Create a test dataset using the generator function
test_dataset = tf.data.Dataset.from_generator(get_test_data, output_types=tf.int32)
# Define a function to pad sequences on the left side
def pad_sequence_left(sequence, max_length):
    pad_length = max_length - tf.shape(sequence)[0]
    padded_sequence = tf.pad(sequence, [[pad_length, 0]], constant_values=tokenizer.pad_token_id)
    return padded_sequence

# Find the maximum sequence length in the dataset
max_sequence_length = test_dataset.map(lambda x: tf.shape(x)).reduce(tf.constant([-1, -1], dtype=tf.int32), tf.maximum)[1]

# Pad sequences on the left side and batch the dataset
test_dataset = test_dataset.map(lambda x: pad_sequence_left(x, max_sequence_length))
test_dataset = test_dataset.batch(batch_size)



In [ ]:
# Initialize a list to store the generated results
test_results = []

# Iterate through the test dataset and generate results
for batch in tqdm.tqdm_notebook(test_dataset, total=steps):
    # Perform inference on the batch
    output_ids = model.generate(input_ids=batch, max_length=80, do_sample=True, top_k=15, top_p=0.7)

    # Decode the generated output
    generated_sentences = [tokenizer.decode(ids.numpy().tolist(),skip_special_tokens = True) for ids in output_ids]

    # Store the generated sentences in the results list
    test_results.extend(generated_sentences)


<ipython-input-34-22081289d755>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm.tqdm_notebook(test_dataset, total=steps):


  0%|          | 0/59 [00:00<?, ?it/s]

In [ ]:
# Define a function to extract the final sentence from a component
def extract_final_sentence(component):
    sentences = component.split('. ')
    final_sentence = sentences[2]
    return final_sentence

# Use a list comprehension to apply the function to the list of components
final_sentences = [extract_final_sentence(component) for component in test_results]

In [ ]:
for idx, oup in enumerate(final_sentences):
    j_list[idx]["output"] = oup

jsonldump(j_list, '/content/drive/MyDrive/tf_kogpt_result_epoch3.json')